# Everyone Can Be a Quantum Open Source Developer! 💖
### Demo for [Open Source Day Summer 2021](https://opensourceday.anitab.org/) 

Dr. Sarah Kaiser |  [@crazy4pi314](twitter.com/crazy4pi314) | 15 July 2021

---

Talk slides/Jupyter Notebook can be found at [github.com/crazy4pi314/osd-2021-qsharp](github.com/crazy4pi314/osd-2021-qsharp)

[![Unitary Fund](https://img.shields.io/badge/Supported%20By-UNITARY%20FUND-yellow.svg?style=flat)](http://unitary.fund)![licence](https://img.shields.io/github/license/crazy4pi314/osd-2021-qsharp)

### Abstract
Quantum computing is an exciting and growing tech field that has adopted open source languages and platforms to address its unique challenges. Just like learning a new package or toolchain in your favorite language, you to can learn quantum computing via open source project and have an impact on the direction of the field! In this workshop, we will cover what in the heck quantum computing actually is, what are the current challenges that we can address with open source tools, and you will write your first quantum program in Python and Q#!

---

#### Installation instructions for running this notebook on your machine can be found [here](https://docs.microsoft.com/quantum/install-guide/python?view=qsharp-preview) or you can run this presentation in your browser [here](bit.ly/osd-2021-qsharp).

# Q\# : Microsoft's quantum programming language

- New open source language that is domain-specific for quantum computing
- Used with the [_Quantum Development Kit_](https://www.microsoft.com/en-us/quantum/development-kit) which provides lots of tools for writing and running your programs.
- Designed to be integrated with a number of languages/platforms like Python and .NET

<h2>The Q# workflow</h2>
<figure style="text-align: center;">
    <img src="media/stack.png" width="80%">
    <caption>
      <br>  
        <strong></strong>
    </caption>
</figure>


### Q# programs can be run from:

- the command line, if built as stand-alone applications
- Python or .NET language programs (C#, F#, etc.) for easy data processing and visualization
- ✨**Jupyter notebooks**✨ with Q# or Python kernels


## Q# Hello World

In [1]:
function Greeting(name : String) : Unit {
    Message($"Hello World! Nice to meet you {name} 💖");
}

Greeting

In [2]:
%simulate Greeting name="Anita"

Hello World! Nice to meet you Anita 💖


()

## Functions + Types in Q#

- Functions are ways to define deterministic, classical calculations.
- Q# is a strongly typed language, with a variety of built-in types and ways for you to define your own!

In [3]:
// Library has the definition of the Complex type we use below.
open Microsoft.Quantum.Math;

In [4]:
// A user defined type for an array of complex numbers.
newtype ComplexArray = (Count : Int, Values : Complex[]);

ComplexArray

In [5]:
// Converts an array of real numbers to a ComplexArray
function AsComplexArray(data : Double[]) : ComplexArray {

    // Start with a new ComplexArray to put your results in
    mutable results = ComplexArray(0, new Complex[0]);
    for item in data {
        // update-and-reassign statement
        set results w/= Values <- results::Values + [Complex(item, 0.)]; 
    }
    
    // Return the results with the Count named item filled out
    return results w/ Count <- Length(results::Values); 
}

AsComplexArray

In [6]:
function ConvertedData() : ComplexArray {
    let data = [1.0, 2.0, 3.0];
    return AsComplexArray(data);
}

ConvertedData

In [7]:
%simulate ConvertedData

ComplexArray((3, [Complex((1, 0)),Complex((2, 0)),Complex((3, 0))]))

## `use` Qubits in Q#

- Qubits are a resource that are requested from the runtime when you need them and returned when you are done.

In [8]:
open Microsoft.Quantum.Measurement;

In [9]:
operation Qrng() : Result {
    use qubit = Qubit();    // Preparing the qubit
    H(qubit);               // Do operation H (prepare |+〉state)
    return MResetZ(qubit);  // Measure and reset qubit
}

Qrng

In [19]:
%simulate Qrng

One

## Operations on Qubits
- PauliX, Hadamard, Controlled Y, Measurement...
- Diagnostics like `DumpMachine` can be helpful to see what is going on

In [20]:
open Microsoft.Quantum.Diagnostics;

operation ApplyQuantumNOT() : Unit {
    use qubit = Qubit();    // Preparing the qubit
    DumpMachine();
    X(qubit);               // Do operation X
    DumpMachine();
    Reset(qubit);
}

ApplyQuantumNOT

In [21]:
%simulate ApplyQuantumNOT

|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖

|0⟩	0 + 0𝑖
|1⟩	1 + 0𝑖

()

## Unit testing in Q#

- Great way to check that what we have typed matches what we wanted 😊

In [22]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;

operation AllocateQubitRegister(numQubits : Int) : Unit {

    Fact(numQubits > 0, "Expected a positive number.");
    use register = Qubit[numQubits];
    ApplyToEach(AssertQubit(Zero, _), register);

    Message("Test passed!");
}

AllocateQubitRegister

In [23]:
%simulate AllocateQubitRegister numQubits=5

Test passed!


()

## Unit testing in Q# cont.

- Helpful to test if _operations with qubits_ are also doing what we want

In [24]:
open Microsoft.Quantum.Diagnostics; 

operation ApplyCNOT(register : Qubit[])
: Unit is Adj + Ctl {
    CNOT(register[0], register[1]);
}

ApplyCNOT

In [25]:
operation ApplyOtherOperation(register : Qubit[])
: Unit is Adj + Ctl {
    within {
        ApplyToEachCA(H, register);
    } 
    apply {
        CNOT(register[1], register[0]);
    }
}

operation CheckThatThisWorks() : Unit {
    AssertOperationsEqualReferenced(2, ApplyCNOT, ApplyOtherOperation);
    Message("It works!");
}

ApplyOtherOperation, CheckThatThisWorks

In [26]:
%simulate CheckThatThisWorks

It works!


()

## ✨**Your turn!**✨ 
> 
> Write a test that checks if the `ApplyPauli(ConstantArray(3, PauliX), _)` operation does the same thing as `ApplyToEachCA(X, _)`. Pssst [Q# library docs](https://docs.microsoft.com/en-us/qsharp/api/qsharp/)

In [27]:
operation CheckPauliXApplication() : Unit {
    AssertOperationsEqualReferenced(3, ApplyPauli(ConstantArray(3, PauliX), _), ApplyToEachCA(X, _));
    Message("It works!");
}

CheckPauliXApplication

In [28]:
%simulate CheckPauliXApplication

It works!


()

## Quantum Development Kit: 
### A toolkit for writing quantum applications, libraries, and new tools
- Notebooks, IDEs, testing, community...
- Intellisense is your friend
<figure style="text-align: left;">
<img src="media/intellisense.png" width="60%">
</figure>

In [29]:
%lsmagic //✨🔮

Name,Summary,Assembly
%azure.connect,Connects to an Azure Quantum workspace or displays current connection status.,Microsoft.Quantum.IQSharp.AzureClient
%azure.execute,Submits a job to an Azure Quantum workspace and waits for completion.,Microsoft.Quantum.IQSharp.AzureClient
%azure.jobs,Displays a list of jobs in the current Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%azure.output,Displays results for a job in the current Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%azure.quotas,Displays a list of quotas for the current Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%azure.status,Displays status for a job in the current Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%azure.submit,Submits a job to an Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%azure.target,Sets or displays the active execution target for Q# job submission in an Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%config,Allows setting or querying configuration options.,Microsoft.Quantum.IQSharp.Kernel
%debug,Steps through the execution of a given Q# operation or function.,Microsoft.Quantum.IQSharp.Kernel


#### How do we know how things scale? 
### Resource estimation!

In [30]:
%estimate CheckThatThisWorks

Metric,Sum,Max
CNOT,6,6
QubitClifford,8,8
R,0,0
Measure,4,4
T,0,0
Depth,0,0
Width,4,4
QubitCount,4,4
BorrowedWidth,0,0


### And now let's look for an anwser with...
# Unstructured Search

<img src="media/CH11_FIG02_struct-unstruct.png" width="80%">

In [31]:
// Start your engines 🏁
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Arrays;

// For later..
function nIterations(nQubits : Int) : Int {
    let nItems = 1 <<< nQubits;                        
    let angle = ArcSin(1. / Sqrt(IntAsDouble(nItems)));
    let total = Round(0.25 * PI() / angle - 0.5);
    return total;
}

nIterations

In [32]:
// Prepares |11111...1>
operation PrepareAllOnes(register : Qubit[]) : Unit is Adj + Ctl {
    ApplyToEachCA(X, register); // HELP 😯
}

// Flip the last qubit if the register is in |11111...1>
operation ReflectAboutAllOnes(register : Qubit[]) : Unit is Adj + Ctl {
    Controlled Z(Most(register), Tail(register));
}

PrepareAllOnes, ReflectAboutAllOnes

In [33]:
// Flip the last qubit if the register is in |++++...+>

operation ReflectAboutUniform(register : Qubit[]) : Unit is Adj + Ctl {
    within {
        Adjoint ApplyToEachCA(H, register); // HELP 😯
        PrepareAllOnes(register);
    } apply {
        ReflectAboutAllOnes(register);
    } 
}

ReflectAboutUniform

In [34]:
// Apply an oracle if the register is in a state 
// representing the number we are looking for (marked item).

operation ReflectAboutMarked(oracle : ((Qubit[], Qubit) => Unit is Adj),
    register : Qubit[]) 
: Unit is Adj {
    use output = Qubit();
    within {
        X(output);
        H(output);
    } apply {
        oracle(register, output); //Is this magic??
    }
}

// THE ORACLE 🔮
operation ApplyOracle(markedItem : Int, inputRegister : Qubit[], outputRegister : Qubit)
: Unit is Adj + Ctl {
    (ControlledOnInt(markedItem, X))(inputRegister, outputRegister);
}

ReflectAboutMarked, ApplyOracle

In [35]:
operation SearchForMarked(
    oracle : ((Qubit[], Qubit) => Unit is Adj), 
    nItems : Int) 
: Int {
    use inputRegister = Qubit[BitSizeI(nItems)];  // What is BitSizeI?
    // Prepare uniform superposition
    ApplyToEach(H, inputRegister);
    // Loop through reflections
    for n in 0..nIterations(BitSizeI(nItems)) - 1 {
        ReflectAboutMarked(oracle, inputRegister);
        ReflectAboutUniform(inputRegister);
    }
    return MeasureInteger(LittleEndian(inputRegister));
}

SearchForMarked

In [37]:
operation RunGroverSearch(nItems : Int, markedItem : Int) : Int {
   let markedOracle = ApplyOracle(markedItem, _, _);
   let foundItem = SearchForMarked(markedOracle, nItems);
   Message($"Marked : {markedItem}, Found : {foundItem}");
   return foundItem;
}

RunGroverSearch

In [41]:
%simulate RunGroverSearch nItems=12 markedItem=3

Marked : 3, Found : 3


3

In [42]:
%estimate RunGroverSearch nItems=20000 markedItem=1298

Metric,Sum,Max
CNOT,73840,73840
QubitClifford,27279,27279
R,0,0
Measure,15,15
T,51688,51688
Depth,33370,33370
Width,29,29
QubitCount,29,29
BorrowedWidth,0,0


## Check out other Q# community projects

<figure style="text-align: left;">
    <img src="media/qsc-projects.png" width="80%">
</figure>

## 👩‍💻Quantum programming resources!👩‍💻

- Q# Documentation: [docs.microsoft.com/quantum](https://docs.microsoft.com/quantum)
- _Learn Quantum Computing with Python and Q\#_ : [bit.ly/qsharp-book](http://www.manning.com/?a_aid=learn-qc-kaiser)
- Community: [qsharp.community](https://qsharp.community/)
    - Q# Community Slack [bit.ly/qsharp-slack](https://bit.ly/qsharp-slack)
    - Women in Quantum Computing and Algorithms (WIQCA): [wiqca.dev](https://wiqca.dev)
    - Quantum Open Source Foundation: [qosf.org](https://www.qosf.org/)
    - Unitary Fund [unitary.fund](https://unitary.fund/)

## ✨**Your turn!** ✨
> 
> Write a function that takes converts angles expressed as radians to a value in degrees (round to nearest degree)
> The [Q# library docs](https://docs.microsoft.com/en-us/qsharp/api/qsharp/) could be helpful ♥

In [ ]:
function RadiansAsDegrees(...) : ...{
    //code that takes radians to degrees
    return ...
}


In [ ]:
%simulate RadiansAsDegrees(3.141592)

## ✨**Your turn!**✨ 
> 
> Write an operation using 2 built-in single qubit operations that rotates a single qubit into the $\frac{1}{\sqrt{2}}|0\rangle - |1\rangle$ state. 
>
> The [Q# library docs](https://docs.microsoft.com/en-us/qsharp/api/qsharp/) could be helpful ♥

In [ ]:
operation PrepareMinusState() : Unit {
    use qubit = Qubit();    // Preparing the qubit
    ...
    DumpMachine();  //Test we got it right
    Reset(qubit);
}

In [ ]:
%simulate PrepareMinusState